In [ ]:
from pathlib import Path
import datetime

import polars as pl

import stock

In [ ]:
len(target_codes)

In [ ]:
target_codes = []

daily_dir = stock.PROJECT_ROOT / "data" / "daily"
financial_dir = stock.PROJECT_ROOT / "data" / "financial"
for financial_csv_path in sorted(financial_dir.glob("*.csv")):
    fdf = stock.kabutan.read_financial_csv(financial_csv_path).filter(
        pl.col("duration") == 3
    ).sort(pl.col("annoounce_date"))
    if len(fdf) == 0:
        continue
    latest_date = fdf["annoounce_date"][-1]

    daily_csv_path = daily_dir / financial_csv_path.name
    if not daily_csv_path.exists():
        continue
    ddf = stock.kabutan.read_data_csv(daily_csv_path)
    before_df = ddf.filter((latest_date - datetime.timedelta(days=365) <= pl.col("date")) & (pl.col("date") < latest_date))
    if len(before_df) < 20:
        continue
    
    # high = before_df["high"][:-5].max()
    # if before_df["high"][-5:].max() < high:  # 新高値ではない場合はスキップ
    #     continue

    mean_volume = before_df["volume"][-10:-5].mean()
    max_volume = before_df["volume"][-5:].max()
    if max_volume < mean_volume * 5:  # 出来高が少ない場合はスキップ
        continue
    if max_volume < 100000: 
        continue

    code = daily_csv_path.stem
    target_codes.append(code)
    print(code, latest_date, mean_volume, max_volume)

In [ ]:
# 出来高の増加とともに新高値をつけている銘柄を探す
target_codes = []
target_date = datetime.date.today()

for csv_path in sorted(daily_dir.glob("*.csv")):
    if csv_path.stem == "0000" or csv_path.stem == "0010":
        continue
    df = stock.kabutan.read_data_csv(csv_path)
    fdf = df.filter(pl.col("date").is_between(target_date - datetime.timedelta(days=365), target_date)).sort(pl.col("date"))
    if len(fdf) < 10:
        continue
    # if fdf["high"].max() != fdf["high"][-1]:
    #     continue

    latest_volume = fdf["volume"][-1]
    mean_volume = fdf["volume"][-6:-1].mean()
    if latest_volume > 100000 and latest_volume > mean_volume * 5:
        stem = csv_path.stem
        target_codes.append(stem)
        print(stem, mean_volume, latest_volume)

In [ ]:
len(target_codes)

In [ ]:
# まずはバックテストで決算直前に出来高が大幅に増えている銘柄を抽出する
